# Trying out sympy module

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sympy as sp
import numpy as np
from Modules.sympy.classes import *
from Modules.sympy.utils import *
from sympy.physics.quantum import Dagger

In [3]:
Spin = RDBasis("\\sigma", 'spin', dim=2)
sx = Spin.project(sp.Matrix([[0, 1], [1, 0]]))
display(sx)
sz = Spin.project(sp.Matrix([[1, 0], [0, -1]]))
display(sz)

\sigma_{1}

\sigma_{3}

In [4]:

a = RDBoson("a", subspace ="b")
ad = RDBoson("{a^\\dagger}", subspace ="b", is_annihilation=False)

hbar = RDsymbol("hbar", order = 0)
omega = RDsymbol("omega", order = 0)
Omega_z = RDsymbol("\\Omega_{z}", order = 0)
g = RDsymbol("g", order = 1)

H = hbar * omega * ad * a + hbar * Omega_z * sp.Rational(1,2) * sz + hbar * g * (a + ad) * sx
H

\Omega_{z}*hbar*\sigma_{3}/2 + g*hbar*(a + {a^\dagger})*\sigma_{1} + hbar*omega*{a^\dagger}*a

In [5]:
Charge = RDBasis("Q", "charge", dim=2)
tx = Charge.project(sp.Matrix([[0, 1], [1, 0]]))

In [6]:
s1_symbols = [RDsymbol("S^1_{}".format(i), order=1) for i in range(4)]
s2_symbols = [RDsymbol("S^2_{}".format(i), order=1) for i in range(4)]
S1 = np.array(s1_symbols, dtype=object).dot(Spin._basis)
S2 = np.array(s2_symbols, dtype=object).dot(Spin._basis)
S = S1*ad + S2*a
S

(S^1_0*\sigma_{0} + S^1_1*\sigma_{1} + S^1_2*\sigma_{2} + S^1_3*\sigma_{3})*{a^\dagger} + (S^2_0*\sigma_{0} + S^2_1*\sigma_{1} + S^2_2*\sigma_{2} + S^2_3*\sigma_{3})*a

In [7]:
H0 = group_by_order(H)[0]
V = group_by_order(group_by_diagonal(H)[False])[1]
V

g*hbar*a*\sigma_{1} + g*hbar*{a^\dagger}*\sigma_{1}

In [8]:
sols = {}
for key, val in group_by_infinite_operators(Spin.project(expand_commutator(Commutator(H0, S) + V).doit())).items():
    if val == 0:
        continue
    display(key)
    display(val)
    sols.update(sp.solve(get_matrix(val), s1_symbols + s2_symbols))



{a^\dagger}

S^1_0*hbar*omega*\sigma_{0} - I*S^1_1*\Omega_{z}*hbar*\sigma_{2} + S^1_1*hbar*omega*\sigma_{1} + I*S^1_2*\Omega_{z}*hbar*\sigma_{1} + S^1_2*hbar*omega*\sigma_{2} + S^1_3*hbar*omega*\sigma_{3} + g*hbar*\sigma_{1}

a

-S^2_0*hbar*omega*\sigma_{0} - I*S^2_1*\Omega_{z}*hbar*\sigma_{2} - S^2_1*hbar*omega*\sigma_{1} + I*S^2_2*\Omega_{z}*hbar*\sigma_{1} - S^2_2*hbar*omega*\sigma_{2} - S^2_3*hbar*omega*\sigma_{3} + g*hbar*\sigma_{1}

In [9]:
group_by_infinite_operators(S.subs(sols))

{1: 0,
 a: I*\Omega_{z}*g*\sigma_{2}/(\Omega_{z}**2 - omega**2) - g*omega*\sigma_{1}/(\Omega_{z}**2 - omega**2),
 {a^\dagger}: I*\Omega_{z}*g*\sigma_{2}/(\Omega_{z}**2 - omega**2) + g*omega*\sigma_{1}/(\Omega_{z}**2 - omega**2)}

In [10]:
for key, val in group_by_infinite_operators(S.subs(sols)).items():
    if val == 0:
        continue
    display(val)
    display(get_matrix(val).simplify())

I*\Omega_{z}*g*\sigma_{2}/(\Omega_{z}**2 - omega**2) - g*omega*\sigma_{1}/(\Omega_{z}**2 - omega**2)

Matrix([
[                      0, g/(\Omega_{z} + omega)],
[-g/(\Omega_{z} - omega),                      0]])

I*\Omega_{z}*g*\sigma_{2}/(\Omega_{z}**2 - omega**2) + g*omega*\sigma_{1}/(\Omega_{z}**2 - omega**2)

Matrix([
[                      0, g/(\Omega_{z} - omega)],
[-g/(\Omega_{z} + omega),                      0]])

In [11]:
get_terms_and_factors(g*Omega_z/(omega**2 - Omega_z**2)*ad)

([\Omega_{z}*g*{a^\dagger}/(-\Omega_{z}**2 + omega**2)],
 [[\Omega_{z}, g, 1/(-\Omega_{z}**2 + omega**2), {a^\dagger}]])

In [12]:
group_by_diagonal(Spin.project(expand_commutator(H + Commutator(H, S) + sp.Rational(1,2) * Commutator(V, S)).doit().subs(sols).simplify().expand().nsimplify()))[False]

RecursionError: maximum recursion depth exceeded

In [20]:
Spin.project(tx)

Q_{1}*\sigma_{3}*\sigma_{0}

In [17]:
tx

Q_{1}